In [7]:
import numpy as np
import yfinance as yf
import warnings
import pandas as pd
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

def ST_labels(data, delta):
    """
    Calculate the stop-loss adjusted label.

    Parameters:
    - data: DataFrame containing historical asset prices.
    - delta: Maximum tolerance level for stop-loss trading.

    Returns:
    - Index of rows where the label is 1 (BUY).
    """
    return data[
        (data["Close"] / data["Close"].shift(1) > 1) & 
        ((data["Low"] / data["Close"].shift(1) - 1) * 100 >= -delta)
    ].index

def moving_average(series: pd.Series, window: int) -> pd.Series:
    return series.rolling(window).mean()

def exponential_moving_average(series: pd.Series, window: int) -> pd.Series:
    return series.ewm(span=window, adjust=False).mean()

def rate_of_change(series: pd.Series, window: int) -> pd.Series:
    shifted = series.shift(window)
    return (series - shifted) / (shifted + 1e-9)

def rsi(series: pd.Series, window: int) -> pd.Series:
    diff = series.diff(1)
    gain = diff.clip(lower=0)
    loss = diff.clip(upper=0).abs()
    avg_gain = gain.rolling(window).mean()
    avg_loss = loss.rolling(window).mean()
    rs = avg_gain / (avg_loss + 1e-9)
    return 100 - (100 / (1 + rs))

def macd_signal(series: pd.Series, short_window: int = 12, long_window: int = 26, signal_window: int = 9):
    ema_short = exponential_moving_average(series, short_window)
    ema_long = exponential_moving_average(series, long_window)
    macd_line = ema_short - ema_long
    signal_line = exponential_moving_average(macd_line, signal_window)
    return macd_line, signal_line

def stochastic_oscillator(high: pd.Series, low: pd.Series, close: pd.Series, window: int = 14):
    highest_high = high.rolling(window).max()
    lowest_low = low.rolling(window).min()
    k = (close - lowest_low) / (highest_high - lowest_low + 1e-9) * 100
    d = k.rolling(3).mean()
    return k, d

def williams_r(high: pd.Series, low: pd.Series, close: pd.Series, window: int = 14):
    highest_high = high.rolling(window).max()
    lowest_low = low.rolling(window).min()
    wr = (highest_high - close) / (highest_high - lowest_low + 1e-9) * -100
    return wr

def bollinger_bands(series: pd.Series, window: int = 20, num_std: float = 2.0):
    m_avg = series.rolling(window).mean()
    m_std = series.rolling(window).std(ddof=0)
    upper_band = m_avg + num_std * m_std
    lower_band = m_avg - num_std * m_std
    return m_avg, upper_band, lower_band

def cci(high: pd.Series, low: pd.Series, close: pd.Series, window: int = 20):
    typical_price = (high + low + close) / 3
    ma = typical_price.rolling(window).mean()
    md = (typical_price - ma).rolling(window).apply(lambda x: np.mean(np.abs(x)))
    cci_val = (typical_price - ma) / (md * 0.015 + 1e-9)
    return cci_val

def create_features_and_labels(df: pd.DataFrame) -> pd.DataFrame:
    df = df.sort_values('Date').reset_index(drop=True)

    features_df = df.copy()

    features_df['NextClose'] = features_df['Close'].shift(-1)
    features_df['Y'] = np.where(features_df['NextClose'] > features_df['Close'], 'BUY', 'SELL')
    features_df.dropna(subset=['NextClose'], inplace=True)

    for delta in [2, 3, 4, 5]:
        buy_index = ST_labels(features_df, delta)
        features_df[f'Y_{delta}'] = np.where(features_df.index.isin(buy_index), 'BUY', 'SELL')

    period_list = list(range(2, 31))
    technical_features = []

    for w in period_list:
        col_sma = f'SMA_{w}'
        features_df[col_sma] = moving_average(features_df['Close'], w)
        technical_features.append(col_sma)

        col_ema = f'EMA_{w}'
        features_df[col_ema] = exponential_moving_average(features_df['Close'], w)
        technical_features.append(col_ema)

        col_roc = f'ROC_{w}'
        features_df[col_roc] = rate_of_change(features_df['Close'], w)
        technical_features.append(col_roc)

        col_rsi = f'RSI_{w}'
        features_df[col_rsi] = rsi(features_df['Close'], w)
        technical_features.append(col_rsi)

        col_wr = f'WR_{w}'
        features_df[col_wr] = williams_r(features_df['High'], features_df['Low'], features_df['Close'], w)
        technical_features.append(col_wr)

        col_cci = f'CCI_{w}'
        features_df[col_cci] = cci(features_df['High'], features_df['Low'], features_df['Close'], w)
        technical_features.append(col_cci)

        col_bb_mid = f'BBmid_{w}'
        col_bb_up = f'BBup_{w}'
        col_bb_dn = f'BBdn_{w}'
        bb_mid, bb_up, bb_dn = bollinger_bands(features_df['Close'], w)
        features_df[col_bb_mid] = bb_mid
        features_df[col_bb_up] = bb_up
        features_df[col_bb_dn] = bb_dn
        technical_features += [col_bb_mid, col_bb_up, col_bb_dn]

        col_sto_k = f'StoK_{w}'
        col_sto_d = f'StoD_{w}'
        sto_k, sto_d = stochastic_oscillator(features_df['High'], features_df['Low'], features_df['Close'], w)
        features_df[col_sto_k] = sto_k
        features_df[col_sto_d] = sto_d
        technical_features += [col_sto_k, col_sto_d]

    macd_line, signal_line = macd_signal(features_df['Close'], 12, 26, 9)
    features_df['MACD'] = macd_line
    features_df['MACD_Signal'] = signal_line
    technical_features += ['MACD', 'MACD_Signal']

    for w in period_list:
        col_vol_sma = f'Volume_SMA_{w}'
        features_df[col_vol_sma] = moving_average(features_df['Volume'], w)
        technical_features.append(col_vol_sma)

        col_vol_ema = f'Volume_EMA_{w}'
        features_df[col_vol_ema] = exponential_moving_average(features_df['Volume'], w)
        technical_features.append(col_vol_ema)

    for w in period_list:
        col_range = f'Range_{w}'
        features_df[col_range] = (features_df['High'] - features_df['Low']).rolling(w).mean()
        technical_features.append(col_range)

        col_close_std = f'CloseStd_{w}'
        features_df[col_close_std] = features_df['Close'].rolling(w).std()
        technical_features.append(col_close_std)

    features_df.dropna(subset=technical_features, inplace=True)

    label_cols = ['Y', 'Y_2', 'Y_3', 'Y_4', 'Y_5', "Close"]
    result_df = features_df[['Date'] + label_cols].join(
        features_df[technical_features]
    )
    return result_df

if __name__ == "__main__":
    start_date = "2020-01-01"
    end_date = "2025-02-13"

    tickers = list(pd.read_csv("./dataset/full_symbol.csv")["Symbol"])

    #tickers = [
    #    'BIIB', 'BA', 'AXP', 'SLB', 'COP', 'AVGO', 'TMO', 'NEE', 'NKE', 'MO',
    #    'WBA', 'QCOM', 'COST', 'ACN', 'CVS', 'T', 'CVX', 'HD', 'DUK', 'CL',
    #    'MMM', 'CSCO', 'BAC', 'LOW', 'BLK', 'MDLZ', 'PM', 'UNH', 'VZ', 'CAT',
    #    'NVDA', 'FDX', 'RTX', 'AIG', 'TMUS', 'INTC', 'PEP', 'TGT', 'GD', 'GS',
    #    'MDT', 'IBM', 'DIS', 'ORCL', 'COF', 'MSFT', 'KO', 'BKNG', 'V', 'LLY',
    #    'ADBE', 'AMZN', 'SBUX', 'BMY', 'MRK', 'XOM', 'F', 'JNJ', 'USB', 'AMT',
    #    'EXC', 'AAPL', 'SPG', 'TXN', 'PFE', 'PG', 'LMT', 'MCD', 'NFLX', 'UNP',
    #    'HON', 'C', 'GOOG', 'AMGN', 'JPM', 'MA', 'CMCSA', 'ABT', 'SO',
    #    'GILD', 'MET', 'MS', 'EMR', 'UPS', 'CRM', 'DHR', 'GOOGL', 'GE', 'WFC',
    #    'WMT'
    #]

    all_results = []

    for ticker in tickers:
        try:
            df = yf.download(
                ticker,
                start=start_date,
                end=end_date,
                group_by='column'
            )

            # MultiIndex인 경우 컬럼명 재설정
            if isinstance(df.columns, pd.MultiIndex):
                df.columns = ['_'.join(col).strip() for col in df.columns.values]

            # 불러온 DataFrame에 필요한 컬럼명 매핑
            possible_cols = list(df.columns)
            rename_dict = {}
            for col in ['Open','High','Low','Close','Volume']:
                alt_name = col + '_' + ticker
                if alt_name in possible_cols:
                    rename_dict[alt_name] = col

            df.rename(columns=rename_dict, inplace=True)

            required_cols = {'Open','High','Low','Close','Volume'}
            if not required_cols.issubset(df.columns):
                continue

            df.reset_index(inplace=True)
            result_df = create_features_and_labels(df)
            result_df['Stock'] = ticker
            all_results.append(result_df)

        except Exception as e:
            print(e)
            continue

    if all_results:
        final_df = pd.concat(all_results, axis=0).reset_index(drop=True)
        print(final_df.head())
    else:
        print("There are No data")

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['AABA']: YFTzMissingError('possibly delisted; no timezone found')
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['AAXN']: YFTzMissingError('possibly delisted; no timezone found')
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['ABMD']: YFTzMissingError('possibly delisted; no timezone found')
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************

        Date     Y   Y_2   Y_3   Y_4   Y_5      Close      SMA_2      EMA_2  \
0 2020-03-26  SELL   BUY   BUY   BUY   BUY  71.242378  68.961697  69.415404   
1 2020-03-27   BUY  SELL  SELL  SELL  SELL  68.526833  69.884605  68.823023   
2 2020-03-30  SELL   BUY   BUY   BUY   BUY  70.406387  69.466610  69.878599   
3 2020-03-31  SELL  SELL  SELL  SELL  SELL  69.389107  69.897747  69.552271   
4 2020-04-01   BUY  SELL  SELL  SELL  SELL  66.773201  68.081154  67.699558   

      ROC_2  ...  CloseStd_26  Range_27  CloseStd_27  Range_28  CloseStd_28  \
0  0.100299  ...     6.749467  3.982606     6.955622  3.893867     7.090737   
1  0.027681  ...     6.442397  4.001934     6.640886  3.941496     6.853163   
2 -0.011734  ...     6.012439  4.019666     6.317907  3.932364     6.518739   
3  0.012583  ...     5.822534  4.041709     5.898961  3.955691     6.205933   
4 -0.051603  ...     5.755815  4.024360     5.744989  3.969334     5.828533   

   Range_29  CloseStd_29  Range_30  CloseStd_30  S

In [8]:
final_df.to_csv("./dataset/FULL.csv", index =False)

In [10]:
!pip install pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 62.4 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [13]:
final_df.to_feather("./dataset/FULL.feather")

In [3]:
import wrds
import pandas as pd
        
db = wrds.Connection(
    wrds_username = 'dbsxo2995',  # 본인 WRDS ID
    wrds_host = 'wrds-cloud.wharton.upenn.edu',
    port = 9737,
    connect_args = {'sslmode':'require'}
)

start_date = '2020-01-01'
end_date   = '2024-12-31'

query = f"""
    SELECT a.date
         , a.permno
         , b.ticker
         , a.prc
         , a.shrout
         , a.ret
         , ABS(a.prc)*a.shrout*1000 AS marketcap
    FROM crsp.dsf AS a
         JOIN crsp.stocknames AS b
              ON a.permno = b.permno
              AND b.namedt <= a.date
              AND a.date <= b.nameenddt
    WHERE b.exchcd = 3                    
      AND ABS(a.prc)*a.shrout >= 10000000
      AND a.date BETWEEN '2010-01-01' AND '2024-09-01'
    ORDER BY a.date, a.permno
"""

nasdaq_over_1B = db.raw_sql(query)

print(nasdaq_over_1B.head())
import pandas as pd
import numpy as np
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
tables = pd.read_html(url)
snp500_df = tables[0]
full_ = np.unique(list(set(nasdaq_over_1B["ticker"]).union(set(snp500_df["Symbol"]))))
pd.DataFrame({"Symbol": full_}).to_csv("./dataset/full_symbol.csv", index = False)

Enter your WRDS username [dbsxo2995]: dbsxo2995
Enter your password: ········


WRDS recommends setting up a .pgpass file.


Create .pgpass file now [y/n]?:  y


Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done
         date  permno ticker    prc     shrout       ret           marketcap
0  2010-01-04   10104   ORCL  24.85  5011220.0  0.013045      124528817000.0
1  2010-01-04   10107   MSFT  30.95  8811000.0   0.01542      272700450000.0
2  2010-01-04   10138   TROW   54.4   258534.0  0.021596       14064249600.0
3  2010-01-04   10324   GENZ  49.35   266100.0  0.006937       13132035000.0
4  2010-01-04   11081   DELL  14.54  1955623.0  0.012535  28434758419.999996


In [16]:
nasdaq = np.unique(nasdaq_over_1B["ticker"])

In [19]:
pd.DataFrame({"Symbol": nasdaq}).to_csv("nasdaq.csv", index = False)

In [24]:
pd.DataFrame( {"Symbol": np.unique((snp500_df["Symbol"]))} ).to_csv("SNP.csv", index = False)

In [6]:
!pip install yfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 15.3 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.4/117.4 kB 27.0 MB/s eta 0:00:00
  Created wheel for peewee: filename=peewee-3.17.9-py3-none-any.whl size=139051 sha256=b8b98b1fe0fa13b351c92d71081243be49c7ae32c62a28c456a79048ed3871ad
  Stored in directory: /root/.cache/pip/wheels/fd/fd/5e/90b9ec95da4fd6c96237b580ce74f89d6bdea547ad151ab5f4
Successfully built peewee

[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [1]:
#

In [1]:
import numpy as np
import yfinance as yf
import warnings
import pandas as pd

In [2]:
df = pd.read_csv("./dataset/FULL.csv")

In [6]:
df[df["Stock"] == "TSLA"].reset_index(drop = True).to_csv("./dataset/TSLA.csv")
df[df["Stock"] == "AAPL"].reset_index(drop = True).to_csv("./dataset/AAPL.csv")
df[df["Stock"] == "MSFT"].reset_index(drop = True).to_csv("./dataset/MSFT.csv")
df[df["Stock"] == "TSLA"].reset_index(drop = True).to_csv("./dataset/TSLA.csv")

df[df["Stock"] == "NVDA"].reset_index(drop = True).to_csv("./dataset/NVDA.csv")
df[df["Stock"] == "META"].reset_index(drop = True).to_csv("./dataset/META.csv")
df[df["Stock"] == "AMZN"].reset_index(drop = True).to_csv("./dataset/AMZN.csv")
df[df["Stock"] == "JPM"].reset_index(drop = True).to_csv("./dataset/JPM.csv")

In [8]:
df[df["Stock"] == "TSLA"].reset_index(drop = True).to_feather("./dataset/TSLA.feather")
df[df["Stock"] == "AAPL"].reset_index(drop = True).to_feather("./dataset/AAPL.feather")
df[df["Stock"] == "MSFT"].reset_index(drop = True).to_feather("./dataset/MSFT.feather")
df[df["Stock"] == "TSLA"].reset_index(drop = True).to_feather("./dataset/TSLA.feather")

df[df["Stock"] == "NVDA"].reset_index(drop = True).to_feather("./dataset/NVDA.feather")
df[df["Stock"] == "META"].reset_index(drop = True).to_feather("./dataset/META.feather")
df[df["Stock"] == "AMZN"].reset_index(drop = True).to_feather("./dataset/AMZN.feather")
df[df["Stock"] == "JPM"].reset_index(drop = True).to_feather("./dataset/JPM.feather")

In [ ]:
df[df["Stock"] == "JPM"].reset_index(drop = True).to_csv("./dataset/JPM.csv")

In [ ]:
df[df["Stock"] == "JPM"].reset_index(drop = True).to_feather("./dataset/JPM.feather")